# Chapitre 5 : Traitement de données avec Pandas

### Importer et mettre les options

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.width', 80)
pd.set_option('display.max_columns', 5)
pd.set_option('precision', 1)
pd.set_option('max_colwidth', 12)

### Charger les données

In [4]:
tableau = pd.read_excel("./data/base-pop-2015-communes.xls")

In [5]:
tableau.head()

,CODGEO,REG,...,C15_POP55P_CS7,C15_POP55P_CS8
0,01001,84,...,140.0,10.0
1,01002,84,...,64.5,0.0
2,01004,84,...,2864.2,244.6
3,01005,84,...,310.0,30.0
4,01006,84,...,24.8,0.0


In [6]:
tableau.shape

(35399, 108)

In [7]:
list(tableau.index)[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
list(tableau.columns)[0:8]

['CODGEO',
 'REG',
 'DEP',
 'LIBGEO',
 'P15_POP',
 'P15_POP0014',
 'P15_POP1529',
 'P15_POP3044']

In [9]:
tableau.loc[0,"DEP"]

'01'

In [10]:
colonne_a_afficher = tableau.columns[0]
tableau[colonne_a_afficher].head()

0    01001
1    01002
2    01004
3    01005
4    01006
Name: CODGEO, dtype: object

### Mettre en forme

In [11]:
colonnes = ['DEP', 'LIBGEO', 'P15_POP', 'P15_POPF', 'P15_POP0014', 
            'P15_POP1529','P15_POP3044', 'P15_POP4559', 
            'P15_POP6074', 'P15_POP7589','P15_POP90P', 
            'P15_F0014', 'P15_F1529', 'P15_F3044', 'P15_F4559',
            'P15_F6074', 'P15_F7589', 'P15_F90P']
tableau = tableau[colonnes]

In [12]:
tableau = tableau.set_index("DEP")

In [13]:
tableau.columns = [
     "Ville", "Total", "Nombre femme", "Nombre 0-14 ans", 
     "Nombre 15-29", "Nombre 30-44", "Nombre 45-59",
     "Nombre 60-74", "Nombre 75-89", "Nombre 90 et +",
     "Nombre femme 0-14 ans", "Nombre femme 15-29",
     "Nombre femme 30-44", "Nombre femme 45-59",
     "Nombre femme 60-74", "Nombre femme 75-89",
     "Nombre femme 90 et +"]

In [14]:
tableau["numeros_departements"] = tableau.index

In [15]:
tableau.loc["01", "noms_departements"] = "Ain"

In [16]:
nouveau_tableau = tableau.drop("noms_departements", axis=1)

### Recodage

In [17]:
def recodage_population(pop):
    if not isinstance(pop, int):
        return None
    if pop < 1000:
        return "Population faible"
    if pop < 50000:
        return "Population moyenne"
    return "Population forte"

In [18]:
recodage = tableau["Total"].apply(recodage_population)
tableau["Total_reco"] = recodage

In [19]:
def calculer_proportion(ligne):
    if pd.isnull(ligne["Total"]) or ligne["Total"]==0:
        return None
    pourcentage = 100 * ligne["Nombre femme"]/ligne["Total"]
    return pourcentage

In [20]:
proportions = tableau.apply(calculer_proportion,axis=1)
tableau["Proportion femmes"] = proportions

In [21]:
def fonction_x2(x):
    return 2*x

In [22]:
tableau["Proportion femmes sup"] = tableau["Proportion femmes"]\
                .apply(lambda x : True if x > 51.45 else False)

In [23]:
for i,l in tableau.head().iterrows():
    print("{} a {} habitants".format(l["Ville"],l["Total"]))

L'Abergement-Clémenciat a 767.0 habitants
L'Abergement-de-Varey a 241.0 habitants
Ambérieu-en-Bugey a 14127.0 habitants
Ambérieux-en-Dombes a 1619.0 habitants
Ambléon a 109.0 habitants


In [24]:
nouveau_tableau = tableau.fillna("donnée absente")

In [25]:
nouveau_tableau = tableau.dropna()

In [26]:
filtre = tableau["Total"] > 100000
echantillon = tableau[filtre]
print(len(echantillon))

42


### Associer des tableaux

In [27]:
donnees_comp = pd.read_excel("data/departements-francais.xls")
donnees_comp["NUMÉRO"] = donnees_comp["NUMÉRO"].astype(str)
donnees_comp = donnees_comp.set_index("NUMÉRO")
noms_departements = donnees_comp["NOM"]
dictionnaire_num_nom = dict(noms_departements)

In [28]:
def associer(x, dic):
    if x in dic:
        return dic[x]
    return None

In [29]:
tableau["noms_départements"] = tableau["numeros_departements"]\
            .apply(associer,args=[dictionnaire_num_nom])
tableau["noms_départements"].head()

DEP
01    None
01    None
01    None
01    None
01    None
Name: noms_départements, dtype: object

In [30]:
donnees_comp = pd.read_excel("data/departements-francais.xls")
donnees_comp["NUMÉRO"] = donnees_comp["NUMÉRO"].astype(str)
donnees_comp["NUMÉRO"] = donnees_comp["NUMÉRO"]\
                    .apply(lambda x : "0"+x if len(x)==1 else x)
donnees_comp = donnees_comp.set_index("NUMÉRO")
donnees_comp = donnees_comp[["NOM","CHEF LIEU"]]

nouveau_tableau = tableau.join(donnees_comp,
                    on="numeros_departements")
nouveau_tableau.head()

,Ville,Total,...,NOM,CHEF LIEU
DEP,,,,,
01,L'Aberge...,767.0,...,Ain,Bourg-en...
01,L'Aberge...,241.0,...,Ain,Bourg-en...
01,Ambérieu...,14127.0,...,Ain,Bourg-en...
01,Ambérieu...,1619.0,...,Ain,Bourg-en...
01,Ambléon,109.0,...,Ain,Bourg-en...


In [31]:
notes = pd.Series([12,14,11,8,19,13,3])
notes

0    12
1    14
2    11
3     8
4    19
5    13
6     3
dtype: int64

In [32]:
notes = pd.DataFrame([
        [12,14],[14,15],[11,18],[8,16],[19,11],[13,14],[3,7]],
        columns = ["notes 1","notes 2"])
notes

,notes 1,notes 2
0,12,14
1,14,15
2,11,18
3,8,16
4,19,11
5,13,14
6,3,7


In [33]:
notes.to_excel("./data/fichier_notes.xls")